# Necessary Installations and Imports

In [1]:
!pip install scikit-fuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894078 sha256=a42241e73b1fdb847589e3f3129c34c3b231f22fd1bb06b3cf56529f8387cc91
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


In [2]:
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import numpy as np

# Automated Pipeline to calculate Fuzy Tip using Python class

In [3]:
class FuzzyTip:

  def __init__(self, inp_temperature, inp_flavour, inp_portion_size, inp_attentiveness, inp_friendliness, inp_speed):

    # Instantiating user inputs
    self.inp_temperature = inp_temperature
    self.inp_flavour = inp_flavour
    self.inp_portion_size = inp_portion_size
    self.inp_attentiveness = inp_attentiveness
    self.inp_friendliness = inp_friendliness
    self.inp_speed = inp_speed

    #declaring inputs and outputs of the Fuzzy Logic System
    # Level 1 inputs
    self.temperature = ctrl.Antecedent(np.linspace(0, 10, 11), 'temperature')
    self.flavour = ctrl.Antecedent(np.linspace(0, 10, 11), 'flavour')
    self.portion_size = ctrl.Antecedent(np.linspace(0, 10, 11), 'portion_size')
    self.attentiveness = ctrl.Antecedent(np.linspace(0, 10, 11), 'attentiveness')
    self.friendliness = ctrl.Antecedent(np.linspace(0, 10, 11), 'friendliness')
    self.speed = ctrl.Antecedent(np.linspace(0, 10, 11), 'speed')
    # Level 1 outputs
    self.food_quality = ctrl.Consequent(np.linspace(0, 10, 11), 'food_quality')
    self.service_quality = ctrl.Consequent(np.linspace(0, 10, 11), 'service_quality')
    # Level 2 inputs
    self.food_quality_for_tip = ctrl.Antecedent(np.linspace(0, 10, 11), 'food_quality_for_tip')
    self.service_quality_for_tip = ctrl.Antecedent(np.linspace(0, 10, 11), 'service_quality_for_tip')
    # Level 2 output
    self.tip = ctrl.Consequent(np.linspace(0, 25, 26), 'tip')

    # membership function
    self.temperature.automf(3)
    self.flavour.automf(3)
    self.portion_size.automf(3)
    self.attentiveness.automf(3)
    self.friendliness.automf(3)
    self.speed.automf(3)
    self.food_quality.automf(3)
    self.service_quality.automf(3)
    self.food_quality_for_tip.automf(3)
    self.service_quality_for_tip.automf(3)
    self.tip.automf(3)


  def fuzzy_food_quality_rules(self):
    food_quality_rules = []
    encoding = {'poor': 0, 'average': 1, 'good': 2}
    metrics = ['poor', 'average', 'good']
    for t in metrics:
        for f in metrics:
            for p in metrics:
                # Define the rule for the current combination
                iter_val = []
                iter_val.append(encoding[t])
                iter_val.append(encoding[f])
                iter_val.append(encoding[p])
                avg_floor = (iter_val[0] + iter_val[1] + iter_val[2]) // 3
                if(avg_floor==0):
                  rule = ctrl.Rule(self.temperature[t] & self.flavour[f] & self.portion_size[p], self.food_quality['poor'])
                elif(avg_floor==1):
                  rule = ctrl.Rule(self.temperature[t] & self.flavour[f] & self.portion_size[p], self.food_quality['average'])
                else:
                  rule = ctrl.Rule(self.temperature[t] & self.flavour[f] & self.portion_size[p], self.food_quality['good'])
                # Add the rule to the list of rules
                food_quality_rules.append(rule)
    return food_quality_rules

  def fuzzy_service_quality_rules(self):
    service_quality_rules = []
    encoding = {'poor': 0, 'average': 1, 'good': 2}
    metrics = ['poor', 'average', 'good']
    for a in metrics:
        for f in metrics:
            for s in metrics:
                # Define the rule for the current combination
                iter_val = []
                iter_val.append(encoding[a])
                iter_val.append(encoding[f])
                iter_val.append(encoding[s])
                avg_floor = (iter_val[0] + iter_val[1] + iter_val[2]) // 3
                if avg_floor == 0:
                  rule = ctrl.Rule(self.attentiveness[a] & self.friendliness[f] & self.speed[s], self.service_quality['poor'])
                elif avg_floor == 1:
                  rule = ctrl.Rule(self.attentiveness[a] & self.friendliness[f] & self.speed[s], self.service_quality['average'])
                else:
                  rule = ctrl.Rule(self.attentiveness[a] & self.friendliness[f] & self.speed[s], self.service_quality['good'])
                # Add the rule to the list of rules
                service_quality_rules.append(rule)
    return service_quality_rules

  def level_one_fuzzy_tree(self):
    #Defining Control System and Simulation for layer one
    food_ctrl = ctrl.ControlSystem(self.fuzzy_food_quality_rules())
    food_sim = ctrl.ControlSystemSimulation(food_ctrl)
    service_ctrl = ctrl.ControlSystem(self.fuzzy_service_quality_rules())
    service_sim = ctrl.ControlSystemSimulation(service_ctrl)
    # Feeding inputs to the first level of fuzzy tree: food quality and service quality control systems
    food_sim.input['temperature'] = self.inp_temperature
    food_sim.input['flavour'] = self.inp_flavour
    food_sim.input['portion_size'] =self.inp_portion_size
    food_sim.compute()
    service_sim.input['attentiveness'] = self.inp_attentiveness
    service_sim.input['friendliness'] = self.inp_friendliness
    service_sim.input['speed'] = self.inp_speed
    service_sim.compute()
    # Retus Control Simulation objects
    return food_sim, service_sim

  def fuzzy_tip_rules(self):
    tip_rules = []
    metrics = ['poor', 'average', 'good']
    for f in metrics:
      for s in metrics:
        if(f=='good' and s =='good'):
          rule = ctrl.Rule(self.food_quality_for_tip[f] & self.service_quality_for_tip[s], self.tip['good'])
        elif((f=='average' or s =='average') and (f!='poor' and s!='poor')):
          rule = ctrl.Rule(self.food_quality_for_tip[f] & self.service_quality_for_tip[s], self.tip['average'])
        else:
          rule = ctrl.Rule(self.food_quality_for_tip[f] & self.service_quality_for_tip[s], self.tip['poor'])
        tip_rules.append(rule)
    return tip_rules

  def level_two_fuzzy_tree(self):
    # Getting Control System Simulation objects from layer one
    food_sim, service_sim = self.level_one_fuzzy_tree()
    # Defining Control System and Simulation for layer two
    tip_ctrl = ctrl.ControlSystem(self.fuzzy_tip_rules())
    tip_sim = ctrl.ControlSystemSimulation(tip_ctrl)
    # Feeding inputs to the second layer of the fuzzy tree: tip control system
    food_quality_output = food_sim.output['food_quality']
    service_quality_output = service_sim.output['service_quality']
    tip_sim.input['food_quality_for_tip'] = food_quality_output
    tip_sim.input['service_quality_for_tip'] = service_quality_output
    tip_sim.compute()
    return tip_sim

  def fuzzy_tip(self):
    tip_sim = self.level_two_fuzzy_tree()
    calculated_tip = tip_sim.output['tip']
    return calculated_tip

# Python Program for Calculating the Tip

In [4]:
class InputOutOfRangeError(Exception):
    def __init__(self, message="Input is out of bounds. It should be in a number in the range 1 to 10."):
        super().__init__(message)


criteria = {0: "Temperature of food", 1: "Flavour of food",
            2: "Portion Size of food", 3: "Attentiveness during service",
            4: "Friendliness during service", 5:"Speed of Service"}
while True:
    # Gather user input for six measures
    print("Please rate your experience in terms of the following Food and Service Quality Measures between 1 to 10 (1=low to 10=high):")
    measures = [0]*6
    i = 0
    while True:
      try:
        measures[i] = float(input(f"{criteria[i]}: "))
        if i==5:
          break
        elif not (1 <= measures[i] <= 10):
          raise InputOutOfRangeError
        else:
          i = i + 1
      except InputOutOfRangeError as e:
        print(e)
        continue
      except ValueError:
        print("Invalid input. Please enter numeric values.")
        continue

    f = FuzzyTip(measures[0], measures[1], measures[2], measures[3], measures[4], measures[5])

    # Calculate the recommended tip
    tip_percentage = round(f.fuzzy_tip(),2)
    print(f"Recommended tip as a percentage is: {tip_percentage}%")

    # Ask the user if they want to enter another set of measures
    another_set = input("Do you want to enter another set of measures? (yes/no): ").strip().lower()
    if another_set != 'yes':
        break


Please rate your experience in terms of the following Food and Service Quality Measures between 1 to 10 (1=low to 10=high):


KeyboardInterrupt: ignored